In [0]:
import numpy as np
from keras.layers import *
from keras.models import *
from keras import *
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from konlpy.tag import Okt

In [11]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 2.8MB/s 
     |████████████████████████████████| 2.7MB 35.5MB/s 


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
from keras.layers import *
from keras.models import *

# RNN

## RNN의 shape

In [2]:
# rnn

# rnn에서는 입력의 shape이 (batch size, time step, indut dim) 이다

x = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15], [16, 17, 18]]])
x

# batch size: 3, time step: 2, indut dim: 3

array([[[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12],
        [13, 14, 15],
        [16, 17, 18]]])

In [4]:
# simple rnn: 2개의 parameter (timesteps, input_dim)

rnn = SimpleRNN(50)(Input(shape=(10, 30)))    # 10이 timestep, 30이 input_dim

## return_sequences

return_sequences
* 디폴트가 False. => "출력 값이 셀의 개수와 동일하다"는 의미. 셀 하나당 하나의 스칼라 값이 리턴.
* return sequences가 True이면 출력값이 (timestep * num of cell) 개이다.

In [6]:
# return_sequences: 디폴트가 False. => "출력 값이 셀의 개수와 동일하다"는 의미. 셀 하나당 하나의 스칼라 값이 리턴
print(rnn.shape)

(?, 50)


In [7]:
# return sequences가 True이면 출력값이 (timestep * num of cell) 개이다.

rnn = SimpleRNN(50, return_sequences=True)(Input(shape=(10, 30)))
print(rnn.shape)

(?, ?, 50)


In [9]:
rnn = SimpleRNN(50, return_sequences=False)(Input(shape=(10, 30)))
print(rnn)

Tensor("simple_rnn_4/TensorArrayReadV3:0", shape=(?, 50), dtype=float32)


# 실습: 챗봇 만들기

In [0]:
from keras import *
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from konlpy.tag import Okt

In [13]:
chatbot_data = pd.read_csv("/content/drive/My Drive/190000 멀티캠퍼스/Data/ChatbotData.csv")
chatbot_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [0]:
question, answer = list(chatbot_data["Q"]), list(chatbot_data["A"])

In [0]:
# 한 문장에서 단어 시퀀스의 최대 개수

max_sequences = 30     # 임의로 30개로 지정

In [0]:
# 임베딩 벡터 차원 지정

embedding_dim = 100    # 임의로 100으로 지정

Embedding을 하는 이유: 데이터 차원을 줄이기 위해서.

In [0]:
# LSTM hidden layer 차원

lstm_hidden_dim = 128    # 임의로 128로 지정

In [0]:
# 정규표현식 필터

RE_FILTER = re.compile("[.,!?\"':;~()]")    # 나중에 없앨 문자들

In [0]:
# 상수 정의

PAD = "<PADDING>"    # 패딩
STA = "<START>"    # 시작
END = "<END>"    # 끝
OOV = "<OOV>"    # 없는 단어

In [0]:
# 태그 문자에 대한 인덱스 정의

PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

In [0]:
# 데이터 타입

ENCODER_INPUT = 0
DECODER_INPUT = 1
DECODER_TARGET = 2

In [24]:
len(question)    # 데이터 크기

11823

In [0]:
# 데이터 일부만 사용

question = question[:100]
answer = answer[:100]

In [26]:
for i in range(10):
  print("Question: " + question[i])
  print("Answer: " + answer[i])
  print()

Question: 12시 땡!
Answer: 하루가 또 가네요.

Question: 1지망 학교 떨어졌어
Answer: 위로해 드립니다.

Question: 3박4일 놀러가고 싶다
Answer: 여행은 언제나 좋죠.

Question: 3박4일 정도 놀러가고 싶다
Answer: 여행은 언제나 좋죠.

Question: PPL 심하네
Answer: 눈살이 찌푸려지죠.

Question: SD카드 망가졌어
Answer: 다시 새로 사는 게 마음 편해요.

Question: SD카드 안돼
Answer: 다시 새로 사는 게 마음 편해요.

Question: SNS 맞팔 왜 안하지ㅠㅠ
Answer: 잘 모르고 있을 수도 있어요.

Question: SNS 시간낭비인 거 아는데 매일 하는 중
Answer: 시간을 정하고 해보세요.

Question: SNS 시간낭비인데 자꾸 보게됨
Answer: 시간을 정하고 해보세요.



## 순서

1. 단어사전 생성
  * 문장 -> 토큰 (글자 or 단어 or 문장) 분리
  * 지금 다루는 데이터는 한국어이므로 konlpy 사용 -> 형태소분석기 (한나눔, 코모란, Okt 등)
  * KoNLPy (자바로 작성되어있음) 사용하기 위해
    
    1) java 설치 (버전 1.7 이상) -> 환경변수 이름: JAVA_HOME, 값: c:\program files\jdk...

    2) jpype1 설치 (파이썬에서 자바를 호출할 수 있도록 하는 라이브러리): 아나콘다 프롬프트에서 pip install jpype 설치경로

    3) KoNLPy 설치: 아나콘다 프롬프트에서 pip install KoNLPy
  * 
2. 전처리
3. 모델 생성
4. 훈련 및 테스트
5. 문장 생성

## 1. 단어사전 생성

In [0]:
# 형태소 분석 함수

def pos_tag(sentences):
  tagger = Okt()
  sentences_pos = []
  for sentence in sentences:
    sentence = re.sub(RE_FILTER, "", sentence)    # 특수문자 제거
    sentence = " ".join(tagger.morphs(sentence))
    sentences_pos.append(sentence)
  return sentences_pos

In [0]:
question = pos_tag(question)
answer = pos_tag(answer)

In [38]:
for i in range(10):
  print("Question: " + question[i])
  print("Answer: " + answer[i])
  print()

Question: 12시 땡
Answer: 하루 가 또 가네요

Question: 1 지망 학교 떨어졌어
Answer: 위로 해 드립니다

Question: 3 박 4일 놀러 가고 싶다
Answer: 여행 은 언제나 좋죠

Question: 3 박 4일 정도 놀러 가고 싶다
Answer: 여행 은 언제나 좋죠

Question: PPL 심하네
Answer: 눈살 이 찌푸려지죠

Question: SD 카드 망가졌어
Answer: 다시 새로 사는 게 마음 편해요

Question: SD 카드 안 돼
Answer: 다시 새로 사는 게 마음 편해요

Question: SNS 맞팔 왜 안 하지 ㅠㅠ
Answer: 잘 모르고 있을 수도 있어요

Question: SNS 시간 낭비 인 거 아는데 매일 하는 중
Answer: 시간 을 정 하고 해보세요

Question: SNS 시간 낭비 인데 자꾸 보게 됨
Answer: 시간 을 정 하고 해보세요



In [0]:
sentences = []
sentences.extend(question)

In [41]:
sentences

['12시 땡',
 '1 지망 학교 떨어졌어',
 '3 박 4일 놀러 가고 싶다',
 '3 박 4일 정도 놀러 가고 싶다',
 'PPL 심하네',
 'SD 카드 망가졌어',
 'SD 카드 안 돼',
 'SNS 맞팔 왜 안 하지 ㅠㅠ',
 'SNS 시간 낭비 인 거 아는데 매일 하는 중',
 'SNS 시간 낭비 인데 자꾸 보게 됨',
 'SNS 보면 나 만 빼고 다 행복 해보여',
 '가끔 궁금해',
 '가끔 뭐 하는지 궁금해',
 '가끔 은 혼자 인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀 난다',
 '가상 화폐 쫄딱 망함',
 '가스 불 켜고 나갔어',
 '가스 불 켜놓고 나온거 같아',
 '가스 비 너무 많이 나왔다',
 '가스 비 비싼데 감기 걸리겠어',
 '가스 비 장난 아님',
 '가장 확실한 건 뭘 까',
 '가족 여행 가기 로 했어',
 '가족 여행 고고',
 '가족 여행 어디 로 가지',
 '가족 있어',
 '가족 관계 알려 줘',
 '가족 끼리 여행 간다',
 '가족 들 보고 싶어',
 '가족 들 이랑 서먹해',
 '가족 들 이랑 서먹해졌어',
 '가족 들 이랑 어디 가지',
 '가족 들 이랑 여행 갈거야',
 '가족 여행 가야 지',
 '가족 이 누구 야',
 '가족 이랑 여행 가려고',
 '가족 한테 스트레스 풀었어',
 '가출 할까',
 '가출 해도 갈 데 가 없어',
 '간만 에 떨리니까 좋더라',
 '간만 에 쇼핑 중',
 '간만 에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장 치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까',
 '감 말랭이 먹고 싶다',
 '감 말랭이 먹어야지',
 '감기 같 애',
 '감기 걸린 것 같아',
 '감기 기운 이 있어',
 '감기 들 거 같 애',
 '감기 가 오려나',
 '감기 약 이 없어',
 '감기 인거 같 애',
 '감미로운 목소리 좋아',
 '감정 이 쓰레기통 처럼 엉망 진창 이야',
 '감정 컨트롤 을 못 하겠어',
 '감정 컨

In [0]:
sentences.extend(answer)

In [43]:
sentences

['12시 땡',
 '1 지망 학교 떨어졌어',
 '3 박 4일 놀러 가고 싶다',
 '3 박 4일 정도 놀러 가고 싶다',
 'PPL 심하네',
 'SD 카드 망가졌어',
 'SD 카드 안 돼',
 'SNS 맞팔 왜 안 하지 ㅠㅠ',
 'SNS 시간 낭비 인 거 아는데 매일 하는 중',
 'SNS 시간 낭비 인데 자꾸 보게 됨',
 'SNS 보면 나 만 빼고 다 행복 해보여',
 '가끔 궁금해',
 '가끔 뭐 하는지 궁금해',
 '가끔 은 혼자 인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀 난다',
 '가상 화폐 쫄딱 망함',
 '가스 불 켜고 나갔어',
 '가스 불 켜놓고 나온거 같아',
 '가스 비 너무 많이 나왔다',
 '가스 비 비싼데 감기 걸리겠어',
 '가스 비 장난 아님',
 '가장 확실한 건 뭘 까',
 '가족 여행 가기 로 했어',
 '가족 여행 고고',
 '가족 여행 어디 로 가지',
 '가족 있어',
 '가족 관계 알려 줘',
 '가족 끼리 여행 간다',
 '가족 들 보고 싶어',
 '가족 들 이랑 서먹해',
 '가족 들 이랑 서먹해졌어',
 '가족 들 이랑 어디 가지',
 '가족 들 이랑 여행 갈거야',
 '가족 여행 가야 지',
 '가족 이 누구 야',
 '가족 이랑 여행 가려고',
 '가족 한테 스트레스 풀었어',
 '가출 할까',
 '가출 해도 갈 데 가 없어',
 '간만 에 떨리니까 좋더라',
 '간만 에 쇼핑 중',
 '간만 에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장 치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까',
 '감 말랭이 먹고 싶다',
 '감 말랭이 먹어야지',
 '감기 같 애',
 '감기 걸린 것 같아',
 '감기 기운 이 있어',
 '감기 들 거 같 애',
 '감기 가 오려나',
 '감기 약 이 없어',
 '감기 인거 같 애',
 '감미로운 목소리 좋아',
 '감정 이 쓰레기통 처럼 엉망 진창 이야',
 '감정 컨트롤 을 못 하겠어',
 '감정 컨

In [49]:
words = []

for sentence in sentences:
  for word in sentence.split():
    words.append(word)

# 길이가 0인 단어는 제거
words = [word for word in words if len(word)>0]

# 중복된 단어 제거
words = list(set(words))

words

['바라요',
 '떨어졌어',
 '물어봐서',
 '연락',
 '까지',
 '친구',
 '뭘',
 '진리',
 '중',
 '해봐요',
 '간식',
 '힘든데',
 '장난',
 '사람',
 '많이',
 '서먹해졌어',
 '시켜',
 '룩',
 '싶어',
 '가스',
 '반',
 '잊고',
 '부모님',
 '같은',
 '먹을까',
 '좋다',
 '편해요',
 '취미',
 '인',
 '모두',
 '강렬한',
 '첫인상',
 '못',
 '간다',
 '당신',
 '키우고',
 '들어올',
 '믿어',
 '감미로운',
 '가자고',
 '있으면',
 '사는',
 '봐요',
 '관리',
 '자의',
 '예요',
 '망가졌어',
 '따뜻하게',
 '됐으면',
 '로',
 '입어볼까',
 '뭐',
 '하',
 '갈거야',
 '보내고',
 '하자',
 '가보세요',
 '운',
 '가네요',
 '것',
 '싫어요',
 '강원도',
 '개인',
 '적',
 '되도록',
 '나를',
 '다시',
 '할',
 '벗어나는',
 '맛있게',
 '부터는',
 '질질',
 '보면',
 '여행',
 '살펴',
 '먹고',
 '자도',
 '눈살',
 '좋아해주세요',
 '행복',
 '맞팔',
 '고민',
 '가난한',
 '감히',
 '닮아서',
 '쇼핑',
 '도',
 '확실한',
 '4일',
 '만나지',
 '집',
 '저',
 '가만',
 '알아차려도',
 '아름다운',
 '기름',
 '어필',
 '하고',
 '먼저',
 '서로',
 '하세요',
 '집어서',
 '함께',
 '좋아요',
 '걸린',
 '개강',
 '했길',
 '될',
 '거',
 '땡',
 '새',
 '궁금해',
 '누굴',
 '싫어하지',
 '알려',
 '방학',
 '목소리',
 '애가',
 '진창',
 '다',
 '패턴',
 '불',
 '안',
 '엉망',
 '떨리는',
 '들',
 '했잖아',
 '보게',
 '박',
 '해보세요',
 '컨트롤',
 '변화',
 '참',
 '봅니다',
 '싶네요',
 '상황',
 '쫄딱',


In [46]:
len(words)

451

In [50]:
# 가장 앞부분에 태그 삽입

# words[:0]    # 출력: []    # 맨 앞을 의미
words[:0] = [PAD, STA, END, OOV]
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '바라요',
 '떨어졌어',
 '물어봐서',
 '연락',
 '까지',
 '친구',
 '뭘',
 '진리',
 '중',
 '해봐요',
 '간식',
 '힘든데',
 '장난',
 '사람',
 '많이',
 '서먹해졌어',
 '시켜',
 '룩',
 '싶어',
 '가스',
 '반',
 '잊고',
 '부모님',
 '같은',
 '먹을까',
 '좋다',
 '편해요',
 '취미',
 '인',
 '모두',
 '강렬한',
 '첫인상',
 '못',
 '간다',
 '당신',
 '키우고',
 '들어올',
 '믿어',
 '감미로운',
 '가자고',
 '있으면',
 '사는',
 '봐요',
 '관리',
 '자의',
 '예요',
 '망가졌어',
 '따뜻하게',
 '됐으면',
 '로',
 '입어볼까',
 '뭐',
 '하',
 '갈거야',
 '보내고',
 '하자',
 '가보세요',
 '운',
 '가네요',
 '것',
 '싫어요',
 '강원도',
 '개인',
 '적',
 '되도록',
 '나를',
 '다시',
 '할',
 '벗어나는',
 '맛있게',
 '부터는',
 '질질',
 '보면',
 '여행',
 '살펴',
 '먹고',
 '자도',
 '눈살',
 '좋아해주세요',
 '행복',
 '맞팔',
 '고민',
 '가난한',
 '감히',
 '닮아서',
 '쇼핑',
 '도',
 '확실한',
 '4일',
 '만나지',
 '집',
 '저',
 '가만',
 '알아차려도',
 '아름다운',
 '기름',
 '어필',
 '하고',
 '먼저',
 '서로',
 '하세요',
 '집어서',
 '함께',
 '좋아요',
 '걸린',
 '개강',
 '했길',
 '될',
 '거',
 '땡',
 '새',
 '궁금해',
 '누굴',
 '싫어하지',
 '알려',
 '방학',
 '목소리',
 '애가',
 '진창',
 '다',
 '패턴',
 '불',
 '안',
 '엉망',
 '떨리는',
 '들',
 '했잖아',
 '보게',
 '박',
 '해보세요',
 '컨트롤

In [90]:
len(words)

455

In [0]:
# 딕셔너리 생성

word_to_index = {w:i for i, w in enumerate(words)}
index_to_word = {i:w for i, w in enumerate(words)}

In [55]:
list(index_to_word.items())[:20]

[(0, '<PADDING>'),
 (1, '<START>'),
 (2, '<END>'),
 (3, '<OOV>'),
 (4, '바라요'),
 (5, '떨어졌어'),
 (6, '물어봐서'),
 (7, '연락'),
 (8, '까지'),
 (9, '친구'),
 (10, '뭘'),
 (11, '진리'),
 (12, '중'),
 (13, '해봐요'),
 (14, '간식'),
 (15, '힘든데'),
 (16, '장난'),
 (17, '사람'),
 (18, '많이'),
 (19, '서먹해졌어')]

## * 참고
Seq2Seq는 **학습** 시 아래와 같은 3개의 데이터가 필요
* 인코더 입력: 12시 땡
* 디코더 입력: start 점심먹으러 가요
* 디코더 출력: 점심먹으러 가요 end

훈련 모델과 예측 모델이 따로 구성되어야 한다.

## 2. 전처리

### 문장 -> 인덱스 변환

In [0]:
def convert_text_to_index(sentences, vocabulary, type):    # type이 0이면 ENCODER_INPUT, 1이면 DECODER_INPUT, 2면 DECODER_TARGET으로 처리
  
  sentences_index = []
  
  for sentence in sentences:

    sentence_index = []

    # 디코더 입력 -> 맨 앞에 START 추가
    if type == DECODER_INPUT:
      sentence_index.extend([vocabulary[STA]])
    
    for word in sentence.split():
      # 사전에 있는 단어면 인덱스 추가
      if vocabulary.get(word) is not None:    # 그 단어에 해당하는 인덱스가 있다면
        sentence_index.extend([vocabulary[word]])    # 해당 인덱스를 넣는다
      else:    # 그 단어에 해당하는 인덱스가 없다면
        sentence_index.extend([vocabulary[OOV]])    # OOV의 인덱스를 넣는다
    
    # 디코더 타겟 -> 맨 뒤에 END 추가
    if type == DECODER_TARGET:
      if len(sentence_index) >= max_sequences:
        sentence_index = sentence_index[:max_sequences-1]+[vocabulary[END]]
      else:
        sentence_index += [vocabulary[END]]
    else:
      if len(sentence_index) >= max_sequences:
        sentence_index = sentence_index[:max_sequences]
    
    #패딩
    sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]    # 빈 자리만큼 PAD로 채우기

    sentences_index.append(sentence_index)

  return np.asarray(sentences_index)

In [75]:
# 인코더 입력 인덱스 변환

x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)    # 문장을 인덱스로 변환해주는 함수
x_encoder    # 전체 문장에 대한 인덱스 변환 결과가 출력    # 예: 12시 땡 -> [268, 113, 0, 0, 0, ..., 0]
# 한 문장에서 단어 시퀀스의 최대 개수: max_sequences

array([[268, 113,   0, ...,   0,   0,   0],
       [233, 442, 274, ...,   0,   0,   0],
       [301, 132,  92, ...,   0,   0,   0],
       ...,
       [256, 374, 116, ...,   0,   0,   0],
       [256, 452,  27, ...,   0,   0,   0],
       [304, 437, 200, ...,   0,   0,   0]])

In [76]:
# 디코더 입력 인덱스 변환

x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)
x_decoder    # 예: start 하루 가 또 가네요

array([[  1, 436, 300, ...,   0,   0,   0],
       [  1, 240, 361, ...,   0,   0,   0],
       [  1,  77, 302, ...,   0,   0,   0],
       ...,
       [  1,  38, 393, ...,   0,   0,   0],
       [  1,  38, 370, ...,   0,   0,   0],
       [  1, 440,  36, ...,   0,   0,   0]])

In [99]:
# 디코더 target(목표) 인덱스 변환

y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)
y_decoder    # 예: 하루 가 또 가네요 end

array([[436, 300, 412, ...,   0,   0,   0],
       [240, 361, 203, ...,   0,   0,   0],
       [ 77, 302, 193, ...,   0,   0,   0],
       ...,
       [ 38, 393, 327, ...,   0,   0,   0],
       [ 38, 370,  61, ...,   0,   0,   0],
       [440,  36, 338, ...,   0,   0,   0]])

### 유의

인코더 입력, 디코더 입력은 임베딩 계층에 들어가는 인덱스 배열

디코더 출력은 원핫인코딩 형식으로 출력

### 원핫 인코딩

In [0]:
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))    # (100, 30, 455)
# 100: 답변 문장의 개수
# 30: 시퀀스 길이
# 455: 전체 단어 개수 (중복 제외)

In [100]:
# 학습할 때는 입력은 인덱스, 출력은 원핫 인코딩 형식
# 디코더의 target을 원핫인코딩으로 변환

for i, sequence in enumerate(y_decoder):
  for k, index in enumerate(sequence):
    one_hot_data[i, k, index] = 1

y_decoder = one_hot_data
y_decoder

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

## 3. 모델 생성

## 4. 훈련 및 테스트

## 5. 문장 생성

In [0]:
############################# 미완 -- 모델 생성부터는 내일 (9/11) 계속하신다고 함 #############################